In [ ]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [ ]:
AGENT_NAME = "s010-multiturn"

In [ ]:
# # (Optional) Create a new task. If you don't create a new task, each message will be sent to a new task. The server will create the task for you.

# import uuid

# TASK_ID = str(uuid.uuid4())[:8]

# rpc_response = client.agents.rpc_by_name(
#     agent_name=AGENT_NAME,
#     method="task/create",
#     params={
#         "name": f"{TASK_ID}-task",
#         "params": {}
#     }
# )

# task = rpc_response.result
# print(task)

In [ ]:
# Test non streaming response
from agentex.types import TextContent

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.send_message(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "Hello what can you do?"},
        "stream": False
    }
)

if not rpc_response or not rpc_response.result:
    raise ValueError("No result in response")

# Extract and print just the text content from the response
for task_message in rpc_response.result:
    content = task_message.content
    if isinstance(content, TextContent):
        text = content.content
        print(text)


In [ ]:
# Test streaming response
from agentex.types.task_message_update import StreamTaskMessageDelta, StreamTaskMessageFull
from agentex.types.text_delta import TextDelta


# The result object of message/send will be a TaskMessageUpdate which is a union of the following types:
# - StreamTaskMessageStart: 
#   - An indicator that a streaming message was started, doesn't contain any useful content
# - StreamTaskMessageDelta: 
#   - A delta of a streaming message, contains the text delta to aggregate
# - StreamTaskMessageDone: 
#   - An indicator that a streaming message was done, doesn't contain any useful content
# - StreamTaskMessageFull: 
#   - A non-streaming message, there is nothing to aggregate, since this contains the full message, not deltas

# Whenn processing StreamTaskMessageDelta, if you are expecting more than TextDeltas, such as DataDelta, ToolRequestDelta, or ToolResponseDelta, you can process them as well
# Whenn processing StreamTaskMessageFull, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

for agent_rpc_response_chunk in client.agents.send_message_stream(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "Hello what can you do?"},
        "stream": True
    }
):
    # We know that the result of the message/send when stream is set to True will be a TaskMessageUpdate
    task_message_update = agent_rpc_response_chunk.result
    # Print oly the text deltas as they arrive or any full messages
    if isinstance(task_message_update, StreamTaskMessageDelta):
        delta = task_message_update.delta
        if isinstance(delta, TextDelta):
            print(delta.text_delta, end="", flush=True)
        else:
            print(f"Found non-text {type(task_message)} object in streaming message.")
    elif isinstance(task_message_update, StreamTaskMessageFull):
        content = task_message_update.content
        if isinstance(content, TextContent):
            print(content.content)
        else:
            print(f"Found non-text {type(task_message)} object in full message.")
